<a href="https://colab.research.google.com/github/SunSlick2/MoveEmails/blob/main/Python_Code_to_Move_PST_Emails_to_Inbox_(Select_Open_PST)_No_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import win32com.client
import pythoncom
import logging
import time
from datetime import datetime, timedelta # Import timedelta, though less critical without detailed date validation
import json
import os
import tkinter as tk
from tkinter import filedialog, messagebox # messagebox is still used for other errors

class PSTtoOSTMigrator:
    def __init__(self):
        self.setup_logging()
        self.migration_report = {
            'start_time': None,
            'end_time': None,
            'total_attempted': 0,
            'total_successful': 0,
            'total_failed': 0,
            'failed_items': [],
            'folder_summary': {}
        }

    def setup_logging(self):
        """Setup comprehensive logging"""
        # Ensure the logs directory exists
        log_dir = "migration_logs"
        os.makedirs(log_dir, exist_ok=True)
        log_filename = os.path.join(log_dir, f'pst_migration_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')

        logging.basicConfig(
            filename=log_filename,
            level=logging.INFO, # Changed to INFO for console output by default, DEBUG for file
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        # Also log to console
        console = logging.StreamHandler()
        console.setLevel(logging.INFO) # Console still INFO for less verbosity unless needed
        formatter = logging.Formatter('%(levelname)s: %(message)s')
        console.setFormatter(formatter)
        logging.getLogger().addHandler(console)

        # Set file handler to DEBUG for detailed logs
        file_handler = logging.FileHandler(log_filename)
        file_handler.setLevel(logging.DEBUG) # More detailed logging to file
        file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        logging.getLogger().addHandler(file_handler)

        logging.info(f"Logging to: {log_filename}")

    # The validate_move method has been completely removed.
    # Its logic is no longer used in process_folder.

    def get_item_signature(self, item):
        """Create a unique signature for reporting/logging"""
        try:
            return {
                'subject': getattr(item, 'Subject', 'N/A'),
                'sent_on': str(getattr(item, 'SentOn', None)) if getattr(item, 'SentOn', None) else None,
                'sender': getattr(item, 'SenderName', 'N/A'),
                'entry_id': getattr(item, 'EntryID', 'N/A'),
                'size': getattr(item, 'Size', 'N/A')
            }
        except Exception as e:
            return {'subject': 'Unknown', 'error': f'Could not get item properties: {e}'}

    def process_folder(self, source_folder, target_inbox_folder, folder_path=""):
        """
        Process all items in a folder and move them to the specified target_inbox_folder.
        This version ensures all emails go directly to the Inbox, without post-move validation.
        """
        success_count = 0
        error_count = 0

        current_folder_path = f"{folder_path}/{source_folder.Name}" if folder_path else source_folder.Name

        logging.info(f"Processing folder: {current_folder_path}")

        try:
            # Create a static list of items to avoid IndexError when modifying the collection
            items_list = list(source_folder.Items)
            item_count = len(items_list) # Use the length of the Python list
            logging.info(f"Found {item_count} items in '{current_folder_path}' to process.")

            # Iterate through the static list in reverse
            for item in reversed(items_list):
                try:
                    # olMail item class is 43
                    if item.Class == 43:
                        self.migration_report['total_attempted'] += 1

                        original_signature = self.get_item_signature(item)
                        logging.info(f"Attempting to move item: {original_signature['subject'][:50]}...")

                        try:
                            item.Move(target_inbox_folder)

                            # Without validation, assume success if .Move() doesn't raise an error
                            success_count += 1
                            self.migration_report['total_successful'] += 1
                            logging.info(f"Successfully moved: {original_signature['subject'][:50]}...")

                        except Exception as move_error:
                            error_count += 1
                            self.migration_report['total_failed'] += 1
                            self.migration_report['failed_items'].append({
                                'folder': current_folder_path,
                                'subject': original_signature['subject'],
                                'error': str(move_error),
                                'original_signature': original_signature
                            })
                            logging.error(f"Move failed for '{original_signature['subject']}': {move_error}", exc_info=True)

                except Exception as item_error:
                    error_count += 1
                    logging.error(f"Error processing item in '{current_folder_path}': {item_error}", exc_info=True)
                    continue

        except Exception as folder_items_error:
            logging.error(f"Error accessing items in folder '{current_folder_path}': {folder_items_error}", exc_info=True)
            # If we can't access items, we count all items (if known) as failed for this folder
            error_count += item_count if 'item_count' in locals() else 0

        # Update folder summary for the source folder
        self.migration_report['folder_summary'][current_folder_path] = {
            'successful_moves_from_this_folder': success_count,
            'failed_moves_from_this_folder': error_count,
            'total_items_in_source_folder': success_count + error_count # This reflects items *attempted* from this folder
        }

        # Process subfolders, *but still move their items to the same target_inbox_folder*
        for subfolder in source_folder.Folders:
            try:
                # Pass the *original* target_inbox_folder to subfolders
                s, e = self.process_folder(subfolder, target_inbox_folder, current_folder_path)
                success_count += s
                error_count += e

            except Exception as subfolder_error:
                logging.error(f"Error processing subfolder '{subfolder.Name}': {subfolder_error}", exc_info=True)
                error_count += 1 # Count the subfolder itself as a failure if it can't be processed

        return success_count, error_count

    def generate_report(self):
        """Generate a comprehensive migration report"""
        report_dir = "migration_reports"
        os.makedirs(report_dir, exist_ok=True)
        report_filename = os.path.join(report_dir, f'migration_report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json')

        self.migration_report['end_time'] = datetime.now().isoformat()
        if self.migration_report['start_time']:
            self.migration_report['duration_seconds'] = (
                datetime.fromisoformat(self.migration_report['end_time']) -
                datetime.fromisoformat(self.migration_report['start_time'])
            ).total_seconds()
        else:
            self.migration_report['duration_seconds'] = 0

        self.migration_report['success_rate'] = (
            self.migration_report['total_successful'] / self.migration_report['total_attempted'] * 100
        ) if self.migration_report['total_attempted'] > 0 else 0

        with open(report_filename, 'w') as f:
            json.dump(self.migration_report, f, indent=2, default=str)

        logging.info(f"Migration report saved to: {report_filename}")

        # Print summary to console
        print(f"\n=== MIGRATION SUMMARY ===")
        print(f"Start Time: {self.migration_report['start_time']}")
        print(f"End Time: {self.migration_report['end_time']}")
        print(f"Duration: {self.migration_report['duration_seconds']:.2f} seconds")
        print(f"Total Items Attempted: {self.migration_report['total_attempted']}")
        print(f"Successful Moves: {self.migration_report['total_successful']}")
        print(f"Failed Moves: {self.migration_report['total_failed']}")
        print(f"Success Rate: {self.migration_report['success_rate']:.2f}%")

        if self.migration_report['failed_items']:
            print(f"\n=== FAILED ITEMS ({len(self.migration_report['failed_items'])}) ===")
            print("Showing first 5 failures (check log file for all details):")
            for i, failed_item in enumerate(self.migration_report['failed_items'][:5], 1):
                print(f"{i}. Subject: {failed_item['subject'][:70]}...")
                print(f"   From Folder: {failed_item['folder']}")
                print(f"   Error: {failed_item['error']}")
                print("-" * 30)

    def select_pst_store(self, namespace):
        """Prompts the user to select an open PST store."""
        pst_stores = []
        for store in namespace.Stores:
            # Check for PST files based on FilePath if available (and it's a PST type)
            # The StoreID check with 00000000...46 is for identifying PSTs specifically
            # Also check if FilePath attribute exists and ends with .pst
            if hasattr(store, 'FilePath') and store.FilePath and store.FilePath.lower().endswith('.pst'):
                pst_stores.append(store)

        if not pst_stores:
            logging.error("No PST files found currently open in Outlook.")
            messagebox.showerror("No PSTs Open", "No PST files are currently open in Outlook. Please open the desired PST file(s) in Outlook first, then run this script.")
            return None

        print("\n--- Select PST File for Migration ---")
        for i, store in enumerate(pst_stores):
            print(f"{i + 1}. {store.DisplayName} (Path: {getattr(store, 'FilePath', 'N/A')})")

        while True:
            try:
                choice = input("Enter the number of the PST file to migrate from: ")
                selected_index = int(choice) - 1
                if 0 <= selected_index < len(pst_stores):
                    return pst_stores[selected_index]
                else:
                    print("Invalid choice. Please enter a number from the list.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    def select_target_inbox(self, namespace):
        """
        Prompts the user to select the target Inbox for an OST-linked account.
        Returns the target_inbox Folder object and the selected Account object.
        """
        accounts = []
        for account in namespace.Accounts:
            # Check if account is Exchange, Office 365, or IMAP (typically uses OST)
            # olExchange = 0, olImap = 5
            if account.AccountType == 0 or account.AccountType == 5:
                accounts.append(account)

        if not accounts:
            logging.error("No Exchange, Office 365, or IMAP accounts found in Outlook.")
            messagebox.showerror("No Accounts Found", "No Exchange, Office 365, or IMAP accounts found in Outlook. Please ensure an appropriate account is configured.")
            return None, None # Return None for both folder and account

        print("\n--- Select Target Inbox (OST-linked Account) ---")
        display_list = [] # To store (Account, Inbox_Folder) pairs for selection
        for i, account in enumerate(accounts):
            try:
                store = account.DeliveryStore
                root_folder = store.GetRootFolder()
                inbox = root_folder.Folders.Item("Inbox")
                display_list.append((account, inbox))
                print(f"{len(display_list)}. {account.DisplayName} (Inbox: {inbox.FolderPath})")
            except Exception as e:
                logging.warning(f"Error accessing Inbox for account '{account.DisplayName}': {e}")
                # This account will not be added to display_list, so it won't be selectable

        if not display_list:
            logging.error("Could not find any selectable Inboxes for OST-linked accounts.")
            messagebox.showerror("No Inboxes Found", "Could not find any selectable Inboxes for OST-linked accounts. Check your Outlook configuration.")
            return None, None

        while True:
            try:
                choice = input("Enter the number of the account whose Inbox you want to use as the target: ")
                selected_index = int(choice) - 1

                if 0 <= selected_index < len(display_list):
                    selected_account, target_inbox = display_list[selected_index]
                    logging.info(f"Target Inbox selected: {target_inbox.FolderPath}")
                    return target_inbox, selected_account # Return both the folder and the account
                else:
                    print("Invalid choice. Please enter a valid number from the list.")
            except ValueError:
                print("Invalid input. Please enter a number.")
            except Exception as e:
                print(f"Error during target inbox selection: {e}. Please try again.")
                logging.error(f"Unexpected error during target inbox selection: {e}", exc_info=True)
                return None, None

    def run_migration(self):
        """Main migration function with comprehensive validation"""
        logging.info("Starting PST to OST migration (all emails to Inbox, no post-move validation).")
        self.migration_report['start_time'] = datetime.now().isoformat()

        pythoncom.CoInitialize()
        outlook = None
        pst_store_obj = None # Will hold the selected PST Store object
        selected_account = None # Will hold the selected Outlook Account object

        try:
            outlook = win32com.client.Dispatch("Outlook.Application")
            namespace = outlook.GetNamespace("MAPI")

            # User selects PST store from those already open in Outlook
            pst_store_obj = self.select_pst_store(namespace)
            if not pst_store_obj:
                logging.error("PST store selection failed. Exiting migration.")
                return False

            pst_store_display_name = pst_store_obj.DisplayName
            pst_file_path = getattr(pst_store_obj, 'FilePath', 'N/A')
            logging.info(f"Selected PST: '{pst_store_display_name}' (Path: {pst_file_path})")

            # User selects target Inbox
            target_inbox, selected_account = self.select_target_inbox(namespace) # Capture both
            if not target_inbox or not selected_account:
                logging.error("Target Inbox selection failed. Exiting migration.")
                return False
            logging.info(f"Selected Target Inbox: {target_inbox.FolderPath}")

            print("\n" + "=" * 50)
            # Use selected_account.DisplayName for better reliability in the confirmation message
            print(f"CONFIRMATION: You are about to MOVE ALL emails from '{pst_store_display_name}'")
            print(f"into the INBOX of '{selected_account.DisplayName}' ('{target_inbox.FolderPath}').")
            print("This action will flatten the PST folder structure into the single target Inbox.")
            print("THIS ACTION CANNOT BE UNDONE. ENSURE YOU HAVE A PST BACKUP.")
            print("=" * 50)

            confirm = input("Type 'CONFIRM' to proceed with the migration: ")
            if confirm.strip().upper() != 'CONFIRM':
                logging.warning("Migration cancelled by user.")
                print("Operation cancelled.")
                return False

            # Process all folders in the selected PST, moving items to the target_inbox
            logging.info(f"Beginning to process folders from '{pst_store_display_name}'.")

            # Start processing from the root folder of the PST (pst_store_obj.GetRootFolder())
            success_count, error_count = self.process_folder(
                pst_store_obj.GetRootFolder(), # Pass the root folder object of the selected PST
                target_inbox
            )

            # Generate final report
            self.generate_report()

            if error_count == 0:
                logging.info("Migration completed successfully with 0 errors!")
            else:
                logging.warning(f"Migration completed with {error_count} errors. Please check the log and report files.")

            return error_count == 0

        except Exception as e:
            logging.error(f"Critical error during migration: {e}", exc_info=True)
            messagebox.showerror("Migration Error", f"A critical error occurred during migration. Check logs for details: {e}")
            return False
        finally:
            # No programmatic PST removal needed as it was never programmatically added.
            # The PST remains open in Outlook as you opened it.

            # Release COM objects
            if outlook:
                del outlook
            pythoncom.CoUninitialize()

# Usage
if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("   PST to OST Email Migration Tool (Select from Open PSTs & Flatten to Inbox)   ")
    print("=" * 60)
    print("This tool requires you to **manually open the desired PST file(s) in Outlook first**.")
    print("It will then list these open PSTs for you to select as the source.")
    print("Emails from all its folders will be MOVED to the INBOX of a selected Outlook account.")
    print("All emails will be moved to the TARGET INBOX, flattening")
    print("the original folder structure of the PST.")
    print("\n" + "=" * 60)

    migrator = PSTtoOSTMigrator()
    success = migrator.run_migration()

    if success:
        print("\n✅ Migration completed successfully!")
    else:
        print("\n❌ Migration completed with errors. Check 'migration_logs' and 'migration_reports' folders for details.")